## Step 1: Calculate timespan for each test

In [9]:
import os
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv, find_dotenv

In [10]:
def __connect_mongo(host, port, username, password, db, userdb = None):
  if username and password:
    if userdb is None:
      userdb = db
    mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, userdb)
    print("MongoUri", mongo_uri)
    conn = MongoClient(mongo_uri)
  else:
    conn = MongoClient(host, port)

    return conn[db]


def read_mongo(db, collection, query={}, host="localhost", port=27017, username=None, password=None, no_id=True, userdb=None):
  db = __connect_mongo(host=host, port=port, username=username, password=password, db=db,userdb=userdb)
  return pd.DataFrame()
  cursor = db[collection].find(query)

  df = pd.DataFrame(list(cursor))

  if no_id:
    del df['_id']

  return df

In [175]:
#try:
load_dotenv('../evaluation_server/.env')
print(os.environ)
#server version
raw_data_df = read_mongo(db=os.environ['dbname'], collection="testResults", host="127.0.0.1", username=os.environ['dbuser'], password=os.environ['dbpass'], userdb="admin")
print("Connected to server")
#except:
    # local version
    #raw_data_df = read_mongo(db="blockchain", collection="testResults")

In [176]:
def __get_timespan(test):
  min_received_at = float('inf')
  max_block_timestamp = 0

  blocks = test["CHAIN"]["blocks"]
  for block in blocks:
    transactions = block["transactions"]
    for transaction in transactions:
      if transaction["sender"] == "THE BLOCKCHAIN":
        continue
      if float(transaction["receivedAt"]) < min_received_at:
        min_received_at = float(transaction["receivedAt"])
      max_block_timestamp = float(block["timestamp"])

  return max_block_timestamp - min_received_at

In [177]:
test_results = []

for i in range(len(raw_data_df)):
  test = raw_data_df.iloc[i]
  timespan = __get_timespan(test)
  test_results.append({
    "test_id": test["TEST_ID"],
    "cipher": test["CIPHER"],
    "cipher_type": test["CIPHER_TYPE"],
    "number_transactions": test["TEST_TRANSACTION_COUNT"],
    "timespan": timespan,
  })


In [178]:
test_results_df = pd.DataFrame(test_results)
test_results_df.head()

,test_id,cipher,number_transactions,timespan
0,11c91773a9454f4a865e345373a543ab,dilithium,1000,5914.65160
1,5a5dccbac2424fa2a41121c3ae9c1c3b,dilithium,1000,7209.34849
2,b6eb77ebca984da0b5766fbb9e31ce45,dilithium,1000,5486.28832
3,ecfe6c8dd97f438885abde944d78c69f,dilithium,1000,6523.08507
4,21d91110c1634a1f9bc998a8f66dc278,dilithium,1000,5709.83667


In [179]:
test_results_path = os.path.join(os.getcwd(), "results", "tests.csv")
test_results_df.to_csv(test_results_path, index=False)

## Step 2: Aggregate test results

In [180]:
cipher_algorithms = set(test_results_df["cipher"].unique())
cipher_types = set(test_results_df["cipher_type"].unique())
numbers_transactions = set(test_results_df["number_transactions"].unique())

In [181]:
aggregated_results = []

for cipher in cipher_algorithms:
  for cipher_type in cipher_types:
    for number_transactions in numbers_transactions:
      # aggregate
      tests = test_results_df.loc[
        (test_results_df.cipher == cipher) &
        (test_results_df.cipher_type == cipher_type) &
        (test_results_df.number_transactions == number_transactions)
      ]
      aggregated_results.append({
        "cipher": cipher,
        "cipher_type": cipher_type,
        "number_transactions": number_transactions,
        "mean": tests["timespan"].mean(),
        "std": tests["timespan"].std(),
      })

In [182]:
aggregated_results_df = pd.DataFrame(aggregated_results)
aggregated_results_df.head()

,cipher,number_transactions,mean,std
0,dilithium,1000,7218.014658,1409.265460
1,dilithium,500,1953.761000,NaN
2,rsa,1000,2625.929465,425.632099
3,rsa,500,NaN,NaN
4,ecc,1000,3380.310179,574.917935


In [183]:
aggregated_results_path = os.path.join(os.getcwd(), "results", "aggregated.csv")
aggregated_results_df.to_csv(aggregated_results_path, index=False)